In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
sns.set()
MONTHS = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
DAYS = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
J_DAYS = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Preprocessing

In [5]:
full_df = pd.read_csv('./raw_data/iec_new_data_reorder.csv')
full_df.head(1)

C:\Users\Chapanin\AppData\Local\Temp\ipykernel_18772\445319331.py:1: DtypeWarning: Columns (1,12,13,14,15,16,17,18,19,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107) have mixed types. Specify dtype option on import or set low_memory=False.
  full_df = pd.read_csv('./raw_data/iec_new_data_reorder.csv')


,SIKUM [SIKUM],no.,building_number,city_code_HHI,city_code_LAMAS,count_reg,city,street,address,lat,...,21:30:00,21:45:00,22:00:00,22:15:00,22:30:00,22:45:00,23:00:00,23:15:00,23:30:00,23:45:00
0,199.341,32,702552276,1932,1200,11,מודיעין מכבים רעות,חרצית,חרצית 32 מודיעין מכבים רעות,31.905868,...,2.7140,2.0600,2.2900,2.2230,2.3500,3.9120,3.4800,2.6770,1.9870,1.1210


In [6]:
dtypes = [np.float32, np.int32, np.int32,  np.int32, np.int32,  np.int32, pd.StringDtype(), pd.StringDtype(), pd.StringDtype(),np.float32,np.float32, pd.StringDtype() ] + [np.float32]*96
dtypes = {c: t for c, t in zip(full_df.columns, dtypes)}
def _map(x):
    if type(x) == str:
        if x == '.':
            return 0
        return x.replace(',', '')
    return x

for column in full_df.columns:
    full_df[column] = [_map(x) for x in full_df[column].values]

full_df = full_df.astype(dtypes)

In [7]:
full_df = full_df.rename({full_df.columns[0]: 'total', 'building_number':'id'}, axis=1)
full_df['ts'] = pd.to_datetime(full_df.date, format='%Y-%m-%d')

In [8]:
full_df['month'] = full_df.ts.dt.month
full_df['day'] = full_df.ts.dt.day
full_df['weekday'] = full_df.ts.dt.day_of_week
full_df['weekday_name'] = pd.Categorical(
    full_df['weekday'].map(lambda x: DAYS[x]), categories=J_DAYS, ordered=True
)
full_df['day_of_year'] = full_df.ts.dt.day_of_year
full_df['week'] = full_df.ts.dt.isocalendar().week
full_df['year'] = full_df.ts.dt.year

In [9]:
full_df.to_pickle('full.pkl')

# Day/Hour df

In [13]:
full_df = pd.read_pickle('full.pkl')
full_df.head(1)

,total,no.,id,city_code_HHI,city_code_LAMAS,count_reg,city,street,address,lat,...,23:30:00,23:45:00,ts,month,day,weekday,weekday_name,day_of_year,week,year
0,199.341003,32,702552276,1932,1200,11,מודיעין מכבים רעות,חרצית,חרצית 32 מודיעין מכבים רעות,31.905869,...,1.987,1.121,2022-01-01,1,1,5,Saturday,1,52,2022


In [33]:
cities = ['מודיעין מכבים רעות', 'באר שבע', 'נתניה'][::-1]
dfs = []
for i, f in enumerate(['hadera_01012022_01012023.xlsx', 'bsv_01012022_01012023.xlsx', 'har_haresha_01012022_01012023.xlsx']):
    df = pd.read_excel(f'raw_data/{f}', skiprows=[1])
    df.columns=['date', 'temperature', 'r_humidity', 'heat_index', 'city']
    df['city'] = cities[i]
    #df.head(1)
    dfs.append(df)
weather_df = pd.concat(dfs)
weather_df['ts'] = pd.to_datetime(weather_df.date, format='%Y-%m-%d %H:%M').dt.floor('H')
weather_df = weather_df.drop(['date'], axis=1)
weather_df = weather_df.groupby(['city', 'ts']).mean().reset_index()

ts1 = pd.Timestamp('2022-06-19T01:00:00.000000000')
ts2 = pd.Timestamp('2022-06-19T02:00:00.000000000')
for city in cities:
    row = weather_df[(weather_df.city == cities[0]) & (weather_df.ts== ts2)].values[0]
    weather_df.loc[len(weather_df.index)] = [city, ts1, row[2], row[3], row[4]] 

weather_df.head(1)

,city,ts,temperature,r_humidity,heat_index
0,באר שבע,2022-01-01,13.15,77.833333,13.15


In [12]:
id_df = full_df[['id', 'count_reg', 'no.', 'city', 'city_code_HHI', 'city_code_LAMAS', 'street', 'address', 'lat', 'lng']].drop_duplicates().set_index('id')
id_df.head(1)

,count_reg,no.,city,city_code_HHI,city_code_LAMAS,street,address,lat,lng
id,,,,,,,,,
702552276,11,32,מודיעין מכבים רעות,1932,1200,חרצית,חרצית 32 מודיעין מכבים רעות,31.905869,35.010933


In [34]:
full_hour_df = full_df.drop(columns=['no.',  'city_code_HHI', 'city_code_LAMAS','address', 'lat', 'lng', 'street', 'ts'])
for hour in range(0,24):
    full_hour_df[f"{hour:02d}"] = full_hour_df[f"{hour:02d}:00:00"] +  full_hour_df[f"{0:02d}:15:00"] +  full_hour_df[f"{0:02d}:30:00"] +  full_hour_df[f"{0:02d}:45:00"]
full_hour_df = full_hour_df.drop(columns=[x for x in full_hour_df.columns if x[0].isnumeric() and len(x)>2])
full_hour_df = pd.melt(full_hour_df, id_vars=[x for x in full_hour_df.columns if not x[0].isnumeric()], var_name='hour', value_name='consumption')
full_hour_df['ts'] = pd.to_datetime(full_hour_df.date + " " + full_hour_df.hour, format='%Y-%m-%d %H')
full_hour_df = full_hour_df.astype({'hour': 'int32'})
full_hour_df = full_hour_df.merge(weather_df, how='left', on=['city', 'ts'])
full_hour_df = full_hour_df.drop(['date'], axis=1)
full_hour_df = full_hour_df.astype({'temperature': 'float32', 'r_humidity': 'float32'})
full_hour_df.head(1)

,total,id,count_reg,city,month,day,weekday,weekday_name,day_of_year,week,year,hour,consumption,ts,temperature,r_humidity,heat_index
0,199.341003,702552276,11,מודיעין מכבים רעות,1,1,5,Saturday,1,52,2022,0,5.037,2022-01-01,10.716666,79.5,10.716667


In [35]:
full_hour_df[(full_hour_df.year == 2022) & (full_hour_df.temperature.isna())]

,total,id,count_reg,city,month,day,weekday,weekday_name,day_of_year,week,year,hour,consumption,ts,temperature,r_humidity,heat_index


In [36]:
cols = [x for x in full_hour_df.columns if x not in ['hour', 'consumption', 'ts']]
full_day_df = full_hour_df[cols].groupby(cols[1:-3], observed=True, as_index=False).mean(numeric_only=False)
full_day_df['ts'] = pd.to_datetime(full_day_df.year.astype(str) + ' ' + full_day_df.month.astype(str)+ ' ' + full_day_df.day.astype(str) , format='%Y %m %d')
full_day_df.head(1)

,id,count_reg,city,month,day,weekday,weekday_name,day_of_year,week,year,total,temperature,r_humidity,heat_index,ts
0,702552276,11,מודיעין מכבים רעות,1,1,5,Saturday,1,52,2022,199.341003,9.863194,84.395836,9.863194,2022-01-01


In [37]:
full_day_df[(full_day_df.year == 2022) & (full_day_df.temperature.isna())]

,id,count_reg,city,month,day,weekday,weekday_name,day_of_year,week,year,total,temperature,r_humidity,heat_index,ts


In [38]:
full_hour_df.to_pickle('hour.pkl')
full_day_df.to_pickle('day.pkl')
id_df.to_pickle('id.pkl')
weather_df.to_pickle('weather.pkl')